In [2]:
### Exploratory data analysis notebook

In [3]:
!pip install graphframes
!pip install nbimporter
!pip install hvac

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.7/154.7 kB 2.1 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.9/155.9 kB 2.2 MB/s eta 0:00:00 0:00:01


In [4]:
import sys
sys.path.append('/home/jovyan/work') # here add notebooks if fails

In [127]:
from pyspark.sql import SparkSession
import nbimporter
from utils.vault_scripts import read_root_token, get_secret_from_vault
from pyspark.sql import functions as F
from pyspark.sql.functions import explode, col, when, lit, expr
from graphframes import GraphFrame

In [6]:
# S3 and Spark configuration

In [7]:
spark = SparkSession.builder.appName("ExpDataAnalysis").getOrCreate()

In [8]:
hadoopConf = spark._jsc.hadoopConfiguration()

In [9]:
AWS_KEY_ID = get_secret_from_vault("aws1", "keyid")
AWS_ACCESS_KEY = get_secret_from_vault("aws2", "accesskey")
AWS_S3_BUCKET = get_secret_from_vault("aws3", "s3bucket")

In [10]:
hadoopConf.set("fs.s3a.access.key", AWS_KEY_ID)
hadoopConf.set("fs.s3a.secret.key", AWS_ACCESS_KEY)
hadoopConf.set("spark.hadoop.fs.s3a.aws.credentials.provider", "org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider")

In [11]:
# Reading all data samples for one NFT collection

In [95]:
s3_path_big = f"s3a://{AWS_S3_BUCKET}/raw/opensea_data/referencedata2/*.json"
s3_path_small = f"s3a://{AWS_S3_BUCKET}/raw/opensea_data/pudgypenguins/*.json"

In [96]:
df_big = spark.read.json(s3_path_big)

In [97]:
df_small = spark.read.json(s3_path_small)

In [99]:
df_asset_events_big = df_big.select(explode(col("asset_events")).alias("event"))

In [101]:
df_asset_events_flat_small = df_small.select(
    col("transaction"),
    col("event_type"),
    col("buyer"),
    col("seller"),
    col("from_address"),
    col("to_address"),
    col("quantity"),
    col("event_timestamp"),
    col("order_hash"),
    col("nft.identifier"),
    col("nft.collection"),
    col("nft.contract"),
    col("payment.decimals"),
    col("payment.quantity").alias("payment_quantity"),
    col("payment.symbol"),
    col("payment.token_address")
)

In [102]:
df_asset_events_flat_big = df_asset_events_big.select(
    col("event.transaction"),
    col("event.event_type"),
    col("event.buyer"),
    col("event.seller"),
    col("event.from_address"),
    col("event.to_address"),
    col("event.quantity"),
    col("event.event_timestamp"),
    col("event.order_hash"),
    col("event.nft.identifier"),
    col("event.nft.collection"),
    col("event.nft.contract"),
    col("event.payment.decimals"),
    col("event.payment.quantity").alias("payment_quantity"),
    col("event.payment.symbol"),
    col("event.payment.token_address")
)

In [112]:
df_asset_events_flat_big.count()

97916

In [113]:
df_asset_events_flat_big.printSchema()

root
 |-- transaction: string (nullable = true)
 |-- event_type: string (nullable = true)
 |-- buyer: string (nullable = true)
 |-- seller: string (nullable = true)
 |-- from_address: string (nullable = true)
 |-- to_address: string (nullable = true)
 |-- quantity: long (nullable = true)
 |-- event_timestamp: long (nullable = true)
 |-- order_hash: string (nullable = true)
 |-- identifier: string (nullable = true)
 |-- collection: string (nullable = true)
 |-- contract: string (nullable = true)
 |-- decimals: long (nullable = true)
 |-- payment_quantity: string (nullable = true)
 |-- symbol: string (nullable = true)
 |-- token_address: string (nullable = true)



In [105]:
df_asset_events_flat_small.count()
# 157470 - cryptopunks
# 97916 - pudgypenguins / referencedata2 > 12177
# 14903 - boredapeyachtclub (SO DATA IS MISSING AFTER MOVING) / reference

12177

In [114]:
grouped_df_big = df_asset_events_flat_big.groupBy("transaction", "identifier", "collection", "event_type").count()
duplicates_df_big = grouped_df_big.filter(F.col("count") > 1)
duplicates_df_big.show()

+--------------------+----------+-------------+----------+-----+
|         transaction|identifier|   collection|event_type|count|
+--------------------+----------+-------------+----------+-----+
|0x2c8c3470effc4c4...|      1987|pudgypenguins|      sale|    8|
|0x4a6dcbeb759e472...|      6197|pudgypenguins|  transfer|    8|
|0x1f1f635b3840388...|      5930|pudgypenguins|      sale|    8|
|0xc42c5aec9e3fd0e...|      5491|pudgypenguins|  transfer|    8|
|0x4369463e4d30b2f...|      6881|pudgypenguins|  transfer|    8|
|0x5fd23c2c0262761...|       168|pudgypenguins|  transfer|    8|
|0xe8ec5c316a52d35...|      3775|pudgypenguins|  transfer|    8|
|0x80728f4c2ad0bbf...|      4105|pudgypenguins|      sale|    8|
|0x95369d2d69eebc4...|      1188|pudgypenguins|  transfer|    8|
|0x88df1674e882cd6...|      3628|pudgypenguins|  transfer|    8|
|0x3ac2fd5f22fec90...|      4145|pudgypenguins|  transfer|    8|
|0x14bd159e97c8a85...|      2640|pudgypenguins|  transfer|    8|
|0x9e29dbedfadb65f...|   

In [110]:
grouped_df_small = df_asset_events_flat_small.groupBy("transaction", "identifier", "collection", "event_type").count()
duplicates_df_small = grouped_df_small.filter(F.col("count") > 1)
duplicates_df_small.show()

+-----------+----------+----------+----------+-----+
|transaction|identifier|collection|event_type|count|
+-----------+----------+----------+----------+-----+
+-----------+----------+----------+----------+-----+



In [115]:
duplicates_df_big.take(5)

[Row(transaction='0x2c8c3470effc4c46979b45be19db812427545567a1f20046803b40d5734b570b', identifier='1987', collection='pudgypenguins', event_type='sale', count=8),
 Row(transaction='0x4a6dcbeb759e472738455c15fdb6ed8e322a2cf373af3ff59960aa137188c7ae', identifier='6197', collection='pudgypenguins', event_type='transfer', count=8),
 Row(transaction='0x1f1f635b3840388c414ba7fb7660ebc0420b0f9025ad3f03d57a1ed17daa0901', identifier='5930', collection='pudgypenguins', event_type='sale', count=8),
 Row(transaction='0xc42c5aec9e3fd0e66385feba929ba1c2384c9665ad2da4e2b9f3e588fe771804', identifier='5491', collection='pudgypenguins', event_type='transfer', count=8),
 Row(transaction='0x4369463e4d30b2ffe77969bfea42679fc56f766c77a8b50460cbedee1eb4ea87', identifier='6881', collection='pudgypenguins', event_type='transfer', count=8)]

In [121]:
filtered_df_big = df_asset_events_flat_big.filter(
    (df_asset_events_flat_big['transaction'] == '0x2c8c3470effc4c46979b45be19db812427545567a1f20046803b40d5734b570b') & 
    (df_asset_events_flat_big['identifier'] == '1987') & 
    (df_asset_events_flat_big['collection'] == 'pudgypenguins') & 
    (df_asset_events_flat_big['event_type'] == 'sale')
)

big_rows = filtered_df_big.collect()

for row in big_rows:
    print(row)


Row(transaction='0x2c8c3470effc4c46979b45be19db812427545567a1f20046803b40d5734b570b', event_type='sale', buyer='0xf84a4eeca2953bf5c16f5fdad2ab85738be66244', seller='0xe5442ae87e0fef3f7cc43e507adf786c311a0529', from_address=None, to_address=None, quantity=1, event_timestamp=1692973835, order_hash='', identifier='1987', collection='pudgypenguins', contract='0xbd3531da5cf5857e7cfaa92426877b022e612cf8', decimals=18, payment_quantity='3940000000000000000', symbol='WETH', token_address='0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2')
Row(transaction='0x2c8c3470effc4c46979b45be19db812427545567a1f20046803b40d5734b570b', event_type='sale', buyer='0xf84a4eeca2953bf5c16f5fdad2ab85738be66244', seller='0xe5442ae87e0fef3f7cc43e507adf786c311a0529', from_address=None, to_address=None, quantity=1, event_timestamp=1692973835, order_hash='', identifier='1987', collection='pudgypenguins', contract='0xbd3531da5cf5857e7cfaa92426877b022e612cf8', decimals=18, payment_quantity='3940000000000000000', symbol='WETH'

In [122]:
filtered_df_small = df_asset_events_flat_small.filter(
    (df_asset_events_flat_small['transaction'] == '0x2c8c3470effc4c46979b45be19db812427545567a1f20046803b40d5734b570b') & 
    (df_asset_events_flat_small['identifier'] == '1987') & 
    (df_asset_events_flat_small['collection'] == 'pudgypenguins') & 
    (df_asset_events_flat_small['event_type'] == 'sale')
)

small_rows = filtered_df_small.collect()

for row in small_rows:
    print(row)

Row(transaction='0x2c8c3470effc4c46979b45be19db812427545567a1f20046803b40d5734b570b', event_type='sale', buyer='0xf84a4eeca2953bf5c16f5fdad2ab85738be66244', seller='0xe5442ae87e0fef3f7cc43e507adf786c311a0529', from_address=None, to_address=None, quantity=1, event_timestamp=1692973835, order_hash='', identifier='1987', collection='pudgypenguins', contract='0xbd3531da5cf5857e7cfaa92426877b022e612cf8', decimals=18, payment_quantity='3940000000000000000', symbol='WETH', token_address='0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2')


In [131]:
distinct_transactions_count_big = df_asset_events_flat_big.groupBy("identifier") \
    .agg(F.countDistinct("transaction").alias("distinct_transaction_count")).orderBy(col("distinct_transaction_count").desc(), col("identifier").asc())

distinct_transactions_count_big.show(truncate=False)

+----------+--------------------------+
|identifier|distinct_transaction_count|
+----------+--------------------------+
|4869      |104                       |
|4372      |62                        |
|2626      |58                        |
|7326      |58                        |
|5712      |53                        |
|7258      |53                        |
|3628      |49                        |
|5104      |48                        |
|180       |41                        |
|6628      |41                        |
|3694      |39                        |
|6030      |39                        |
|4481      |38                        |
|533       |37                        |
|2480      |36                        |
|2804      |34                        |
|4542      |34                        |
|6093      |34                        |
|6763      |34                        |
|4019      |32                        |
+----------+--------------------------+
only showing top 20 rows



In [132]:
distinct_transactions_count_small = df_asset_events_flat_small.groupBy("identifier") \
    .agg(F.countDistinct("transaction").alias("distinct_transaction_count")).orderBy(col("distinct_transaction_count").desc(), col("identifier").asc())

distinct_transactions_count_small.show(truncate=False)

+----------+--------------------------+
|identifier|distinct_transaction_count|
+----------+--------------------------+
|4869      |104                       |
|4372      |62                        |
|2626      |58                        |
|7326      |58                        |
|5712      |53                        |
|7258      |53                        |
|3628      |49                        |
|5104      |48                        |
|180       |41                        |
|6628      |41                        |
|3694      |39                        |
|6030      |39                        |
|4481      |38                        |
|533       |37                        |
|2480      |36                        |
|2804      |34                        |
|4542      |34                        |
|6093      |34                        |
|6763      |34                        |
|4019      |32                        |
+----------+--------------------------+
only showing top 20 rows



In [94]:
# Select the relevant columns
distinct_rows = df_asset_events_flat_small.select("transaction", "identifier", "collection", "event_type").distinct()

# Count the number of distinct rows
distinct_count = distinct_rows.count()

# Show the count
print(f"Number of distinct rows: {distinct_count}")

Number of distinct rows: 12177


In [20]:
# First check time frame that is included by single API call

In [70]:
min_max_timestamps = df_asset_events_flat.agg(
    F.from_unixtime(F.min("event_timestamp")).alias("min_event_timestamp"),
    F.from_unixtime(F.max("event_timestamp")).alias("max_event_timestamp")
)
min_max_timestamps.show()

+-------------------+-------------------+
|min_event_timestamp|max_event_timestamp|
+-------------------+-------------------+
|2023-07-01 00:14:47|2023-09-30 23:48:35|
+-------------------+-------------------+



In [20]:
# Difference between event types
# Sale is the ultimate action that reflects a completed transaction.
# Offer and Order can initiate the process leading to a Sale
# Transfer may occur both before and after a Sale, 
# potentially as part of a wash trading scheme where assets are moved around to give a false impression of market activity.

In [21]:
df_grouped_by_event = df_asset_events_flat.groupBy("event_type").agg(
    F.count("*").alias("count")
)
df_grouped_by_event.show()

+----------+-----+
|event_type|count|
+----------+-----+
|      sale| 4970|
|  transfer|19533|
+----------+-----+



In [22]:
# Payment decomposition
# payment_quantity = 23630000000000000000
# decimals = 18
# Amount in WETH = 23630000000000000000 /10^18 = 23.63

In [37]:
df_count_by_symbol = df_asset_events_flat.groupBy("symbol").agg(
    F.count("*").alias("record_count")
).orderBy(col("record_count").desc())
df_count_by_symbol.show()

+------+------------+
|symbol|record_count|
+------+------------+
|  NULL|       19533|
|  WETH|        3669|
|   ETH|        1287|
|  USDC|          14|
+------+------------+



In [23]:
# Wash trading analysis sample

In [24]:
# 1. Rapid transfers between a small group of accounts (often involving back-and-forth transfers of the same asset for a set of assets).
# 2. Minimal time differences between buys and sells (or transfers).
# 3. No real change in ownership, as the seller and buyer may be the same person, or colluding accounts.
# 4. Repeated behavior over a period, showing a cycle of transfers or sales without any actual long-term holding.

In [25]:
# Build a Transaction Graph

# Nodes: Each node represents a wallet or address.
# Edges: Each edge represents an event (sale or transfer).
# In sale events, the edge connects the seller to the buyer.
# In transfer events, the edge connects the sender to the receiver.

In [26]:
# 1. Rapid transfers between a small group of accounts - Graph Library

In [27]:
df_transfers = df_asset_events_flat.filter(col("event_type") == "transfer").select(
    col("from_address").alias("from"),
    col("to_address").alias("to"),
    col("identifier").alias("nft_id"), 
    col("collection").alias("collection"),
    col("event_timestamp").alias("timestamp"),
    lit(None).cast("double").alias("price"),
    lit(None).cast("string").alias("payment_symbol")
)

df_sales = df_asset_events_flat.filter(col("event_type") == "sale").select(
    col("seller").alias("from"),
    col("buyer").alias("to"),
    col("identifier").alias("nft_id"),
    col("collection").alias("collection"),
    col("event_timestamp").alias("timestamp"),
    (col("payment_quantity") / expr("pow(10, decimals)")).alias("price"),
    col("symbol").alias("payment_symbol")
)

df_edges = df_transfers.unionByName(df_sales, allowMissingColumns=True)
df_edges.take(5)

[Row(from='0xdbfd76af2157dc15ee4e57f3f942bb45ba84af24', to='0xdbfd76af2157dc15ee4e57f3f942bb45ba84af24', nft_id='9386', collection='boredapeyachtclub', timestamp=1692520391, price=None, payment_symbol=None),
 Row(from='0x2e9a18d66f2fc535497cfb395d7f1bcb6746e582', to='0xe0b6e70261db8ef63df421662dbba26ae8470a5d', nft_id='3045', collection='boredapeyachtclub', timestamp=1692520127, price=None, payment_symbol=None),
 Row(from='0xc67db0df922238979da0fd00d46016e8ae14cecb', to='0xea5b1f2f29d89dd959b11cf7d28065f1f25cf697', nft_id='8902', collection='boredapeyachtclub', timestamp=1692519851, price=None, payment_symbol=None),
 Row(from='0xa8d87df83755179e579967014adf733be7c8023d', to='0xdb5485c85bd95f38f9def0ca85499ef67dc581c0', nft_id='9860', collection='boredapeyachtclub', timestamp=1692518927, price=None, payment_symbol=None),
 Row(from='0xdbfd76af2157dc15ee4e57f3f942bb45ba84af24', to='0x501036f867924cdfb7d903205ccb8539bf286bd0', nft_id='4232', collection='boredapeyachtclub', timestamp=169251

In [28]:
df_vertices = df_edges.select( col("from").alias("id")) \
    .union(df_edges.select(col("to").alias("id"))).distinct() # Nodes between participants

df_edges_graph = df_edges.select(
    col("from").alias("src"),
    col("to").alias("dst"),
    col("nft_id"),
    col("timestamp"),
    col("price")
)

g = GraphFrame(df_vertices, df_edges_graph)

/usr/local/spark/python/pyspark/sql/dataframe.py:168: UserWarning: DataFrame.sql_ctx is an internal property, and will be removed in future releases. Use DataFrame.sparkSession instead.
  warnings.warn(


In [32]:
transaction_counts = g.edges.groupBy("src", "nft_id").count() \
    .withColumnRenamed("src", "id")

transaction_counts = transaction_counts.filter(col("count") > 5)  # threshold should be set to some other value

filtered_participants = g.vertices.join(transaction_counts, "id", "inner").orderBy(col("count").desc())

filtered_participants.show(truncate=False)

+------------------------------------------+------+-----+
|id                                        |nft_id|count|
+------------------------------------------+------+-----+
|0x29469395eaf6f95920e59f858042f0e28d98a20b|2779  |229  |
|0x29469395eaf6f95920e59f858042f0e28d98a20b|7491  |129  |
|0x29469395eaf6f95920e59f858042f0e28d98a20b|5691  |115  |
|0x29469395eaf6f95920e59f858042f0e28d98a20b|9860  |108  |
|0x29469395eaf6f95920e59f858042f0e28d98a20b|3569  |88   |
|0x29469395eaf6f95920e59f858042f0e28d98a20b|847   |76   |
|0x29469395eaf6f95920e59f858042f0e28d98a20b|4429  |62   |
|0x29469395eaf6f95920e59f858042f0e28d98a20b|4858  |58   |
|0x29469395eaf6f95920e59f858042f0e28d98a20b|8872  |53   |
|0x29469395eaf6f95920e59f858042f0e28d98a20b|6671  |53   |
|0x29469395eaf6f95920e59f858042f0e28d98a20b|3055  |50   |
|0x29469395eaf6f95920e59f858042f0e28d98a20b|6399  |49   |
|0x29469395eaf6f95920e59f858042f0e28d98a20b|8199  |46   |
|0x29469395eaf6f95920e59f858042f0e28d98a20b|3815  |42   |
|0x29469395eaf